# NorSand Plot

### Libraries

#### General

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


#### Plaxis

In [2]:
from plxscripting.easy import *
localhostport_s = 10000
s_i, g_i = new_server('localhost', localhostport_s, password='')

localhostport_s = 10001
s_o, g_o = new_server('localhost', localhostport_s, password='')

### Functions

#### Extract data

In [6]:
def extract_data(phases,x,y):
    data = {'sigma_mi':[], 'evoid_ratio':[],'psi':[],'eta':[],'eta/Mtheta':[],'theta':[],
            'Dp':[],'K':[],'G':[],'iplas':[],'e':[],'q':[],'p':[],
            'sx':[],'sy':[],'sz':[],'sxy':[],'ea':[],'eps_v':[],
            'eps_1':[],'eps_2':[],'eps_3':[],'phase':[],'s1':[],'s2':[],'s3':[]}
    phaseid=-1
    for phase in phases:
        phaseid +=1
        for step in phase.Steps:
            try:
                
                # state parameters from NorSand
                data['sigma_mi'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.StateParameters[0],(x,y)))
                data['evoid_ratio'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.StateParameters[1],(x,y)))
                data['psi'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.StateParameters[2],(x,y)))
                data['eta'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.StateParameters[3],(x,y)))
                data['eta/Mtheta'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.StateParameters[4],(x,y)))
                data['theta'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.StateParameters[5],(x,y)))
                data['Dp'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.StateParameters[6],(x,y)))
                data['K'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.StateParameters[7],(x,y)))
                data['G'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.StateParameters[8],(x,y)))
                data['iplas'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.StateParameters[9],(x,y)))

                # General variables
                data['e'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.VoidRatio,(x,y)))
                data['q'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.DeviatoricStress,(x,y)))
                data['p'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.MeanEffStress,(x,y))*-1)
                data['sx'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.SigxxE,(x,y))*-1)
                data['sy'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.SigyyE,(x,y))*-1)
                data['sz'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.SigzzE,(x,y))*-1)
                data['s1'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.SigmaEffective1,(x,y))*-1)
                data['s2'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.SigmaEffective2,(x,y))*-1)
                data['s3'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.SigmaEffective3,(x,y))*-1)
                data['sxy'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.Sigxy,(x,y))*-1)
                data['ea'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.Eps1,(x,y))*-100)
                data['eps_v'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.TotalVolumetricStrain,(x,y))*100)
                data['eps_1'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.Eps1,(x,y))*-1)
                data['eps_2'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.Eps2,(x,y))*-1)
                data['eps_3'].append(g_o.getsingleresult(step,g_o.ResultTypes.Soil.Eps2,(x,y))*-1)
                data['phase'].append(phaseid)
            except:
                print('could not download data from one step')
                continue



    data = {'data':pd.DataFrame(data)}
    data['Gamma'] = g_i.Materials[1].User5.value
    data['lambda_e'] = g_i.Materials[1].User6.value
    data['Mtc'] = g_i.Materials[1].User7.value
    data['N'] = g_i.Materials[1].User8.value
    data['xitc'] = g_i.Materials[1].User9.value
    data['H0'] = g_i.Materials[1].User10.value
    data['Hpsi'] = g_i.Materials[1].User11.value
    data['R'] = g_i.Materials[1].User12.value
    data['S'] = g_i.Materials[1].User13.value
    data['psi_0'] = g_i.Materials[1].User14.value
    return data


def calculate_Mi(psi,M,N,Xi,Mtc,method='Dafalias'):
    if method=='Dafalias':
        if psi>=0:
            return M * (1 - N*Xi/Mtc*psi)
        else:
            return M * (1 + N*Xi/Mtc*psi)
    else:
        print('only Dafalias method was added')


def extra_calculos(data):
    data['data']['Mtheta'] = data['data']['eta']/data['data']['eta/Mtheta']
    data['data']['xi_i'] = data['xitc']/(1-data['lambda_e']*data['xitc']/data['Mtc'])
    data['data']['Mi'] = data['data'].apply(lambda x: calculate_Mi(x.psi,x.Mtheta,data['N'],x.xi_i,data['Mtc']),axis=1)
    return data

#### Plot data

In [4]:
def qp_surface(Pp,M):
    p = np.linspace(0,Pp,10000)
    q = np.sqrt(M**2 * (p*(Pp-p)))

    p = np.append(p,p[::-1])
    q = np.append(q,-q[::-1])

    return p,q

def M_line(M):
    p = np.linspace(0,10000,2)
    q = M*p

    return p,q

# Change of coordinates to deviatoric plane
def deviatoriccoords(sigma1,sigma2,sigma3):
    x = (sigma3-sigma2)*(1/np.sqrt(2))
    y = (1/np.sqrt(6))*(sigma1*2 -sigma3-sigma2)
    return x,y


def Critical_Surface(Mtc):
    theta = np.linspace(np.radians(-30),np.radians(30),1000)
    M = Mtc * (1 - Mtc/(3+Mtc)*np.cos(3*theta/2+np.pi/4))
    thetapack = np.concatenate((theta,theta+np.pi/3,theta+np.pi*2/3,theta+np.pi,theta+np.pi*4/3,theta+np.pi*5/3))
    Mpack = np.concatenate((M,M[::-1],M,M[::-1],M,M[::-1]))
    return thetapack, Mpack

def polates_a_cartesianas(radios,angulos,normal=(1,1,1)):
    normal = np.array(normal)
    normal = normal/np.linalg.norm(normal)

    v1 = np.array([1,-1,0])
    v1 = v1 / np.linalg.norm(v1)

    v2 = np.cross(normal,v1)
    v2 = v2 / np.linalg.norm(v2)
    
    x_prime = radios * np.cos(angulos)
    y_prime = radios * np.sin(angulos)

    puntos_3d = np.outer(x_prime,v1)+np.outer(y_prime,v2)
    return puntos_3d[:,0],puntos_3d[:,1],puntos_3d[:,2]


def plot_deviatoric_CS(Mtc,p):
    theta, r = Critical_Surface(Mtc)
    r = r * p
    x,y,z = polates_a_cartesianas(r,theta)
    x,y = deviatoriccoords(x,y,z)
    return x,y

def custom_geomspace(start, stop, num, factor=2.0):
    ratios = np.logspace(0, np.log(stop/start)/np.log(factor), num, base=factor)
    return start * ratios


def yield_surface_orig(M,p_im):

    p = np.linspace(0.01,p_im/np.exp(-1))
    eta = M * (1 + np.log(p_im/p))

    q = eta * p

    return p,q

def yield_surface_truncated(M,p_im,pcap):

    p = np.linspace(pcap,p_im/np.exp(-1))
    eta = M * (1 + np.log(p_im/p))

    q = eta * p

    p = np.append(pcap,p)
    q = np.append(0,q)

    return p,q

### Extract data from PLAXIS

In [16]:
data = extract_data([g_o.Phases[-2],g_o.Phases[-1]],0.05,0.05)


### Save it

In [ ]:
data = extra_calculos(data)
data['data'].to_csv('Arcesio_Undrained.csv')

### Load data and make extra calculations

In [ ]:
data['data'] = pd.read_csv('Arcesio_Undrained.csv')
data['data']['pcap'] = pcap = data['data']['sigma_mi']*np.exp(1*min(data['data']['psi'])*data['xitc']/data['data']['Mi'])

# ea_q = data['data'].loc[data['data']['phase']==data['data']['phase'].unique()[-1]]['ea'].values[0]
# data['data']['ea'] = data['data']['ea'] - ea_q 

### Plot

In [ ]:
# Limits
# -------------------------------------------------------------------
pmax = 500
pmin_e = 1
pmax_e = 1000
qmax =500
qmax_dev = qmax*1.5
emax = 1.1
emin = 0.8
eamax = 10
s1max = 60*5
scale_true = 20
scale = pmax/20
# -------------------------------------------------------------------

for i in range(len(data['data']['p'].values)):

    plt.figure(figsize=(10,10))

    # ----------------------------------------------------------------------------------------
    # AXES 1 - p' VS q
    # ----------------------------------------------------------------------------------------
    ax1 = plt.subplot(221)

    ax1.set_xlabel('Mean effective stress, $p\'$ [kPa]')
    ax1.set_ylabel('Deviatoric stress, $q$ [kPa]')
    ax1.axvline(data['data']['p'].values[i],color='gray',linestyle='dotted')
    ax1.grid(color='gray',alpha=0.2)
    ax1.set_xlim(0,pmax)
    ax1.set_ylim(0,qmax)

    
    # Mtc line
    p_M, q_M = M_line(data['Mtc'])
    ax1.plot(p_M,q_M,color='k',label='$M_{tc}$ ='+str(np.round(data['Mtc'],decimals=2)))

    # Mtheta line
    p_M, q_M = M_line(data['data']['Mtheta'].values[i])
    ax1.plot(p_M,q_M,color='k',label='$M_{\\theta}$ = '+str(np.round(data['data']['Mtheta'].values[i],decimals=2)),linestyle='--')

    # stress path
    ax1.plot(data['data']['p'].values[:i+1],data['data']['q'].values[:i+1],color='r',alpha=0.5,zorder=10)
    ax1.scatter(data['data']['p'].values[i],data['data']['q'].values[i],color='r',zorder=10)
    
    # original yield surface
    p_ys,q_ys = yield_surface_orig(data['data']['Mi'].values[i],data['data']['sigma_mi'].values[i])
    ax1.plot(p_ys,q_ys,color='mediumvioletred',linestyle='--')

    # truncated yield surface
    pcap = data['data']['sigma_mi'].values[i]*np.exp(1*min(data['data']['psi'])*data['xitc']/data['data']['Mi'].values[i])
    
    p_ys,q_ys = yield_surface_truncated(data['data']['Mi'].values[i],data['data']['sigma_mi'].values[i],pcap)
    ax1.plot(p_ys,q_ys,color='mediumvioletred',linewidth=2,label='Yield surface ($p_{im}$ = '+str(np.round(data['data']['sigma_mi'].values[i],decimals=1))+' kPa)')

    ax1.legend(loc='upper left')


    # ----------------------------------------------------------------------------------------
    # AXES 2 - q VS eps_yy
    # ----------------------------------------------------------------------------------------
    ax2 = plt.subplot(222)

    ax2.set_xlabel('Axial strain, $\\varepsilon_{yy}$ [%]')
    ax2.set_ylabel('Deviatoric stress, $q$ [kPa]')
    ax2.grid(color='gray',alpha=0.2)
    ax2.set_xlim(0,eamax)
    ax2.set_ylim(0,qmax)

    # stress path
    ax2.plot(data['data']['ea'].values[:i+1],data['data']['q'].values[:i+1],color='r',alpha=0.5)
    ax2.scatter(data['data']['ea'].values[i],data['data']['q'].values[i],color='r')

    # ----------------------------------------------------------------------------------------
    # AXES 3 - e VS p'
    # ----------------------------------------------------------------------------------------
    ax3 = plt.subplot(224)

    ax3.set_xlabel('Mean effective stress, $p\'$ [kPa]')
    ax3.set_ylabel('Void ratio, $e$ [-]')
    ax3.grid(color='gray',alpha=0.2)
    ax3.set_xlim(pmin_e,pmax_e)
    ax3.set_ylim(emin,emax)
    ax3.set_xscale('log')

    # stress path
    ax3.plot(data['data']['p'].values[:i+1],data['data']['evoid_ratio'].values[:i+1],color='r',alpha=0.5,zorder=10)
    ax3.scatter(data['data']['p'].values[i],data['data']['evoid_ratio'].values[i],color='r',zorder=10)

    # CSL line
    ax3.plot([1,10000],[data['Gamma'],data['Gamma']-data['lambda_e']*np.log(10000)],color='k',label='Critical state line')

    # state parameter
    if data['data']['psi'].values[i]>0:
        ax3.arrow(data['data']['p'].values[i],data['data']['evoid_ratio'].values[i],
                0,-data['data']['psi'].values[i],color='limegreen',label='$\\psi$ = '+str(np.round(data['data']['psi'].values[i],decimals=2)),head_width=0.00001)
    else:
        ax3.arrow(data['data']['p'].values[i],data['data']['evoid_ratio'].values[i],
                0,-data['data']['psi'].values[i],color='firebrick',label='$\\psi$ = '+str(np.round(data['data']['psi'].values[i],decimals=2)),head_width=0.00001)


    ax3.legend(loc='upper right')

    # ----------------------------------------------------------------------------------------
    # AXES 4 - DEVIATORIC PLOT
    # ----------------------------------------------------------------------------------------

    ax4 = plt.subplot(223)
    ax4.set_axis_off()
    
    # Select maximum values
    xs1, ys1 = deviatoriccoords(4,0,0)
    xs2, ys2 = deviatoriccoords(0,4,0)
    xs3, ys3 = deviatoriccoords(0,0,4)

    # Consider maximum values in axis ranges
    ax4.axis([xs2*0.84*1.1*s1max,xs3*0.84*1.1*s1max,min(ys2,ys3)*1*1.1*s1max,ys1*0.84*1.1*s1max])

    # Names of axis
    ax4.text((xs3-xs2)*0.04*s1max,ys1*0.8*s1max,'$\sigma_{x}$')
    ax4.text(xs3-(xs3-xs2)*0.15*s1max*3,min(ys2,ys3)*s1max,'$\sigma_{y}$')
    ax4.text(xs2+(xs3-xs2)*0.1025*s1max*3,min(ys2,ys3)*s1max,'$\sigma_{z}$')

    # Principal axis
    ax4.arrow(0,0,xs3*0.8*s1max,ys3*0.8*s1max,head_width=(-ys3/20*s1max),edgecolor='k',facecolor='k',zorder=19)
    ax4.arrow(0,0,xs2*0.8*s1max,ys2*0.8*s1max,head_width=(-ys3/20*s1max),edgecolor='k',facecolor='k',zorder=19)
    ax4.arrow(0,0,xs1*0.8*s1max,ys1*0.8*s1max,head_width=(-ys3/20*s1max),edgecolor='k',facecolor='k',zorder=19)

    # gray surface
    # ------------
    x_CS, y_CS = plot_deviatoric_CS(data['Mtc'],pmax)
    ax4.fill(x_CS,y_CS,edgecolor='k',facecolor='whitesmoke',zorder=19)
    ax4.plot(x_CS,y_CS,color='gray',zorder=21)

    for j in np.linspace(0,1,5):
        x_CS, y_CS = plot_deviatoric_CS(data['Mtc'],pmax*j)
        ax4.plot(x_CS,y_CS,color='lightgrey',linestyle='--',zorder=19)

    for j in np.linspace(0,len(x_CS)-1,13):
        j = int(j)
        xr = [0,x_CS[j]]
        yr = [0,y_CS[j]]
        ax4.plot(xr,yr,color='lightgrey',zorder=19,linestyle='dotted')
    # ------------

    # Black line (surface with same p')
    x_CS, y_CS = plot_deviatoric_CS(data['Mtc'],data['data']['p'].values[i])
    ax4.plot(x_CS,y_CS,color='k',zorder=20)

    # Coloured surface
    # ------------
    x_ys, y_ys = plot_deviatoric_CS(data['data']['Mi'].values[i],p_ys[q_ys.argmax()])
    ax4.fill(x_ys,y_ys,edgecolor='mediumvioletred',facecolor='palevioletred',zorder=21)

    x_yscs, y_yscs = plot_deviatoric_CS(data['data']['eta'].values[i],data['data']['p'].values[i])
    ax4.fill(x_yscs,y_yscs,edgecolor='mediumvioletred',facecolor='pink',zorder=22)
    ax4.plot(x_yscs,y_yscs,color='mediumvioletred',zorder=23)

    for j in [0.1,0.3,0.5,0.7,0.9]:

        p = 0+(p_ys[q_ys.argmax()]-0)*j
        
        Mm = data['data']['eta'].values[i] +(data['data']['Mi'].values[i]-data['data']['eta'].values[i])*j
        # print(p,q)
        # print('max',max(p_ys))
        x_ysf, y_ysf = plot_deviatoric_CS(Mm,p)
        ax4.plot(x_ysf,y_ysf,color='mediumvioletred',zorder=21,linestyle='--')

    for j in np.linspace(0,len(x_ys)-1,13):
        j = int(j)
        xr = [x_yscs[j],x_ys[j]]
        yr = [y_yscs[j],y_ys[j]]
        ax4.plot(xr,yr,color='mediumvioletred',zorder=22,linestyle='dotted')

    # ------------

    # stress path
    x_state,y_state =deviatoriccoords(data['data']['sx'].values[i]*np.sqrt(3/2),
                                        data['data']['sy'].values[i]*np.sqrt(3/2),
                                        data['data']['sz'].values[i]*np.sqrt(3/2))
    
    ax4.scatter(x_state,y_state,color='r',zorder=25)


    plt.savefig('output\\'+str(i)+'.jpg',bbox_inches='tight')
    plt.show()
    # break